In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar100, imdb
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Conv2DTranspose,MaxPooling1D, Embedding,Input, Conv1D, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from tensorflow.keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils
from keras.preprocessing import sequence

In [2]:
top_words = 1000
# Now we split our data-set into training and test data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


model = Sequential()
model.add(Embedding(1000, 32, input_length=max_words))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
# model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu',name="d1"))

model.add(Dense(1, activation='sigmoid', name="d2"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           32000     
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 64)           6208      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
d1 (Dense)                   (None, 256)               4096256   
_________________________________________________________________
d2 (Dense)                   (None, 1)                 2

In [4]:
# Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
196/196 [==============================] - 36s 32ms/step - loss: 0.4741 - accuracy: 0.7452 - val_loss: 0.3338 - val_accuracy: 0.8551
Epoch 2/10
196/196 [==============================] - 6s 30ms/step - loss: 0.2912 - accuracy: 0.8770 - val_loss: 0.3014 - val_accuracy: 0.8706
Epoch 3/10
196/196 [==============================] - 6s 29ms/step - loss: 0.2575 - accuracy: 0.8944 - val_loss: 0.3177 - val_accuracy: 0.8617
Epoch 4/10
196/196 [==============================] - 6s 30ms/step - loss: 0.2237 - accuracy: 0.9089 - val_loss: 0.3019 - val_accuracy: 0.8760
Epoch 5/10
196/196 [==============================] - 6s 30ms/step - loss: 0.1804 - accuracy: 0.9286 - val_loss: 0.3294 - val_accuracy: 0.8694
Epoch 6/10
196/196 [==============================] - 6s 30ms/step - loss: 0.1267 - accuracy: 0.9531 - val_loss: 0.4014 - val_accuracy: 0.8503
Epoch 7/10
196/196 [==============================] - 6s 30ms/step - loss: 0.0721 - accuracy: 0.9765 - val_loss: 0.4972 - val_accuracy: 0.854

In [5]:
scores = model.evaluate(X_test, y_test, verbose=1)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 4s 5ms/step - loss: 0.7377 - accuracy: 0.8560
Accuracy: 85.60%


In [6]:
teacher_WO_Softmax = Model(model.input, model.get_layer('d1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
# val_dense = teacher_WO_Softmax.predict(X_val)


s1Train=train_dense[:,:256]


In [7]:
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=(500,))
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile('adam',loss=['binary_crossentropy','mse'],loss_weights=[0.5,0.5],metrics=['accuracy'])

    return gan

In [8]:
def build_sdiscriminator():
    
    input2 = Input(shape=(256,),name='input')
    inp=Dense(128)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    conv3 = Dense(128,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    conv3 = Dense(256,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    b_n=Dropout(0.25)(b_n)
    conv3 = Dense(256,activation='relu')(b_n)
    b_n = BatchNormalization()(conv3)
    conv4 = Dense(512,activation='relu')(b_n)
    b_n = BatchNormalization()(conv4)
    dense = Dense(1,activation='sigmoid')(b_n)
    output2=Dense(256,activation='relu')(b_n)
    disc = Model(input2,[dense,output2])          
    disc.compile(Adam(learning_rate=0.0001,beta_1=0.5),loss=['binary_crossentropy','mse',],metrics=['accuracy'])


    return disc

In [29]:
def define_model(name):
  model = Sequential()
  model.add(tf.keras.layers.Embedding(1000, 10, input_length=500,name=name))
  model.add(Dropout(0.2))
  model.add(Conv1D(128, 3,strides=2, activation='relu'))
  model.add(MaxPooling1D())
  model.add(Conv1D(64, 3,strides=2, activation='relu'))
  model.add(MaxPooling1D())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu',name=name+"req"))
	# compile model
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  return model

In [30]:
define_model("s1").summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Embedding)               (None, 500, 10)           10000     
_________________________________________________________________
dropout_20 (Dropout)         (None, 500, 10)           0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 249, 128)          3968      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 124, 128)          0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 61, 64)            24640     
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 30, 64)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1920)             

In [31]:
def training(generator,discriminator,gan,features,epo=20):
    BATCH_SIZE = 128
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
            
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [32]:
s1=define_model("s1")
d1 = build_sdiscriminator()
gan1 = build_gan(s1,d1)


In [33]:
s1=training(s1,d1,gan1,s1Train,20)

195/195 [==============================] - 16s 66ms/step
Epoch: 1 | Discriminator Loss: 1.107090 | Generator Loss: 0.309384 | 
195/195 [==============================] - 13s 67ms/step
Epoch: 2 | Discriminator Loss: 0.880819 | Generator Loss: 0.277014 | 
195/195 [==============================] - 13s 68ms/step
Epoch: 3 | Discriminator Loss: 0.832897 | Generator Loss: 0.300230 | 
195/195 [==============================] - 13s 66ms/step
Epoch: 4 | Discriminator Loss: 0.818006 | Generator Loss: 0.325710 | 
195/195 [==============================] - 13s 66ms/step
Epoch: 5 | Discriminator Loss: 0.815093 | Generator Loss: 0.322453 | 
195/195 [==============================] - 13s 66ms/step
Epoch: 6 | Discriminator Loss: 0.809131 | Generator Loss: 0.336379 | 
195/195 [==============================] - 13s 66ms/step
Epoch: 7 | Discriminator Loss: 0.799256 | Generator Loss: 0.339582 | 
195/195 [==============================] - 13s 66ms/step
Epoch: 8 | Discriminator Loss: 0.794923 | Generator Lo

In [34]:
o1=s1.get_layer("s1req").output

output=Activation('relu')(o1)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)
# output4=Activation('softmax')(output3)
mm=Model(s1.get_layer("s1").input,output3)
my_weights=model.get_layer('d2').get_weights()
mm.get_layer('d11').set_weights(my_weights)

In [35]:
i=0
for l in mm.layers[:len(mm.layers)-1]:
    l.trainable=False

In [36]:
adam = Adam(0.0002,0.5,0.999)

mm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
batch_size = 128
mm_history=mm.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
196/196 [==============================] - 3s 13ms/step - loss: 3.8112 - accuracy: 0.5550 - val_loss: 0.3936 - val_accuracy: 0.8267
Epoch 2/10
196/196 [==============================] - 2s 13ms/step - loss: 1.7863 - accuracy: 0.6451 - val_loss: 0.4106 - val_accuracy: 0.8291
Epoch 3/10
196/196 [==============================] - 2s 11ms/step - loss: 0.8892 - accuracy: 0.7163 - val_loss: 0.3650 - val_accuracy: 0.8425
Epoch 4/10
196/196 [==============================] - 2s 11ms/step - loss: 0.6006 - accuracy: 0.7566 - val_loss: 0.3553 - val_accuracy: 0.8454
Epoch 5/10
196/196 [==============================] - 2s 11ms/step - loss: 0.5031 - accuracy: 0.7752 - val_loss: 0.3748 - val_accuracy: 0.8420
Epoch 6/10
196/196 [==============================] - 2s 11ms/step - loss: 0.4612 - accuracy: 0.7865 - val_loss: 0.3785 - val_accuracy: 0.8458
Epoch 7/10
196/196 [==============================] - 2s 11ms/step - loss: 0.4538 - accuracy: 0.7910 - val_loss: 0.3882 - val_accuracy: 0.8420

In [38]:
# 1 student evaluate model
_, acc = mm.evaluate(X_test, y_test, verbose=1)
print('> %.2f' % (acc * 100.0))

782/782 [==============================] - 3s 4ms/step - loss: 0.3908 - accuracy: 0.8426
> 84.26


In [39]:
mm.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1_input (InputLayer)        [(None, 500)]             0         
_________________________________________________________________
s1 (Embedding)               (None, 500, 10)           10000     
_________________________________________________________________
dropout_23 (Dropout)         (None, 500, 10)           0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 249, 128)          3968      
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 124, 128)          0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 61, 64)            24640     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 30, 64)            0   

# Feed Forward Method

In [40]:
ss1= define_model("s1")

In [41]:
batch_size = 128
mm_history=ss1.fit(X_train, s1Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1)

Epoch 1/10
196/196 [==============================] - 3s 14ms/step - loss: 0.2416 - accuracy: 0.2262
Epoch 2/10
196/196 [==============================] - 3s 13ms/step - loss: 0.2127 - accuracy: 0.2794
Epoch 3/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1395 - accuracy: 0.3309
Epoch 4/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1158 - accuracy: 0.3422
Epoch 5/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1112 - accuracy: 0.3474
Epoch 6/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1076 - accuracy: 0.3510
Epoch 7/10
196/196 [==============================] - 2s 13ms/step - loss: 0.1054 - accuracy: 0.3548
Epoch 8/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1032 - accuracy: 0.3624
Epoch 9/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1017 - accuracy: 0.3666
Epoch 10/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1004 - accura

In [42]:
o1=ss1.get_layer("s1req").output

output=Activation('relu')(o1)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)
mm=Model(ss1.get_layer("s1").input,output3)
my_weights=model.get_layer('d2').get_weights()
mm.get_layer('d11').set_weights(my_weights)

In [43]:
for l in mm.layers[:len(mm.layers)-1]:
    l.trainable=False

In [44]:
adam = Adam(0.0002,0.5,0.999)

mm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
batch_size = 128
mm_history=mm.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
196/196 [==============================] - 3s 13ms/step - loss: 0.5271 - accuracy: 0.8648 - val_loss: 0.4311 - val_accuracy: 0.8543
Epoch 2/10
196/196 [==============================] - 2s 11ms/step - loss: 0.3624 - accuracy: 0.8598 - val_loss: 0.3450 - val_accuracy: 0.8522
Epoch 3/10
196/196 [==============================] - 2s 11ms/step - loss: 0.3325 - accuracy: 0.8592 - val_loss: 0.3375 - val_accuracy: 0.8539
Epoch 4/10
196/196 [==============================] - 2s 11ms/step - loss: 0.3180 - accuracy: 0.8680 - val_loss: 0.3358 - val_accuracy: 0.8540
Epoch 5/10
196/196 [==============================] - 2s 11ms/step - loss: 0.3161 - accuracy: 0.8652 - val_loss: 0.3348 - val_accuracy: 0.8541
Epoch 6/10
196/196 [==============================] - 2s 11ms/step - loss: 0.3157 - accuracy: 0.8661 - val_loss: 0.3346 - val_accuracy: 0.8538
Epoch 7/10
196/196 [==============================] - 2s 11ms/step - loss: 0.3136 - accuracy: 0.8683 - val_loss: 0.3349 - val_accuracy: 0.8541

In [46]:
# ff student evaluate model
_, acc = mm.evaluate(X_test, y_test, verbose=1)
print('> %.2f' % (acc * 100.0))

782/782 [==============================] - 3s 4ms/step - loss: 0.3352 - accuracy: 0.8550
> 85.50
